<a href="https://colab.research.google.com/github/wcjb/Deep-learn/blob/master/20190321%E9%94%80%E5%94%AE%E5%88%86%E7%BA%A7%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('/home/wcjb/下载/SKC排名.xlsx',sheet_name=0)

In [0]:
import multiprocessing as mp
df1 = df.copy()
# 品类数
type = list(set(df1.品类))
# 站点列表
station = list(set(df1.区域))

# 唯一码与对应全国销量总和

In [0]:
df2 = df1.sort_values(by='销量(sum)',ascending=False)
df2.head()

,区域,店铺id,店铺,品类,id,销量,销量(sum),库存,全国畅销款,区域畅销款,店铺排名,区域排名,全国排名
426713,郑州区域,HN04,郑州正弘城店,2,75063654370028000000,122,8733,1464,NaN,NaN,NaN,NaN,NaN
253537,北京区域,BJ09,北京西直门凯德店,2,75063654370028000000,50,8733,1025,NaN,NaN,NaN,NaN,NaN
461624,青岛区域,RZ01,日照万象汇店,2,75063654370028000000,18,8733,570,NaN,NaN,NaN,NaN,NaN
215425,上海区域,SH10,上海三林印象城店,2,75063654370028000000,18,8733,207,NaN,NaN,NaN,NaN,NaN
535971,贵阳区域,XF22,贵阳南国花锦店,2,75063654370028000000,48,8733,2688,NaN,NaN,NaN,NaN,NaN


In [0]:
# 各品类的和
sp = {}
for i in type:
    sp[i] = df2[df2.品类==i]['销量'].sum()

In [0]:
id = list(set(df2.id))

df4 = pd.DataFrame()
df4['id'] = df2['id'].values
df4['品类'] = df2['品类'].values
df4['销量(sum)'] = df2['销量(sum)'].values
df4.drop_duplicates(subset='id',keep='first',inplace=True)

# 全国单品各品类占比
df4.index = range(df4.shape[0])
for i in tqdm(df4.index):
    df4.at[i,'累计销量占比'] = df4.at[i,'销量(sum)']/sp[df4.at[i,'品类']]

for i in tqdm(type):
    df5 = df4[df4['品类']==i]
    for i in df5.index:
        df4.at[i,'累计销量占比'] = df5['累计销量占比'].cumsum()[i]

100%|██████████| 16/16 [00:08<00:00,  1.92it/s]


In [0]:
df_type = df4
for i in tqdm(df_type.index):
    if df_type.at[i,'累计销量占比'] < 0.4:
        df_type.at[i,'全国畅销款'] = 'S'
    elif df_type.at[i,'累计销量占比']<0.7:
	    df_type.at[i,'全国畅销款'] = 'A'
    else:
        df_type.at[i,'全国畅销款'] = 'B'
	

100%|██████████| 53863/53863 [00:01<00:00, 43377.01it/s]


In [0]:
df_type['全国排名'] = df_type.index+1
df_type.head()

,id,品类,销量(sum),累计销量占比,全国畅销款,全国排名
0,75063654370028000000,2,8733,0.014300,S,1
1,5302155930035000000,2,7405,0.026425,S,2
2,5143656410003980000,9,6713,0.037616,S,3
3,75404815510042000000,9,6252,0.072649,S,4
4,75442708780039500000,2,5831,0.035972,S,5


In [0]:
df5 = pd.DataFrame()
df5['区域'] = df2['区域'].values
df5['id'] = df2['id'].values
df5['品类'] = df2['品类'].values
df5['销量'] = df2['销量'].values
df5.drop_duplicates(subset='id',keep='first',inplace=True)

In [0]:
for i in tqdm(station):
    df9 = df5[df5['区域'] == i]  
    for j in tqdm(df9.index):
        df5.at[j,'销量(区域)'] = df9[df9['id']==df5.at[j,'id']]['销量'].sum()


100%|██████████| 17/17 [01:05<00:00,  4.01s/it]


In [0]:
sta = {}
for i in tqdm(station):
    df8 = df5[df5['区域']==i]
    step = {}
    for j in type:
        step[j] = df8[df8.品类==j]['销量(区域)'].sum()
    sta[i] = step

100%|██████████| 17/17 [00:00<00:00, 40.60it/s]


In [0]:


df5.index = range(df5.shape[0])
for i in tqdm(df5.index):
    df5.at[i,'累计销量占比'] = df5.at[i,'销量(区域)']/sta[df5.at[i,'区域']][df5.at[i,'品类']]

for j in tqdm(station):
    df6 = df5[df5['区域']==j]
    n = 1
    for m in df6.index:
        df5.at[m,'区域排名'] = n
        n += 1
    for k in tqdm(type):
        df7 = df6[df6['品类']==k]
        for i in df7.index:
            df5.at[i,'累计销量占比'] = df7['累计销量占比'].cumsum()[i]

100%|█████████▉| 53813/53863 [00:01<00:00, 34289.89it/s]/home/wcjb/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
100%|██████████| 17/17 [00:08<00:00,  1.90it/s]


In [0]:
df5

,区域,id,品类,销量,销量(区域),累计销量占比,区域排名
0,郑州区域,75063654370028000000,2,122,122.0,0.075123,1.0
1,成都区域,5302155930035000000,2,10,10.0,0.007812,1.0
2,北京区域,5143656410003980000,9,20,20.0,0.049261,1.0
3,南京区域,75404815510042000000,9,6,6.0,0.010582,1.0
4,南京区域,75442708780039500000,2,44,44.0,0.020706,2.0
5,杭州区域,75327794820058000000,2,34,34.0,0.032978,1.0
6,重庆区域,75442911630059600000,2,6,6.0,0.003030,1.0
7,青岛区域,2011326170003980000,2,4,4.0,0.002701,1.0
8,沈阳区域,75133223310035000000,2,24,24.0,0.017817,1.0
9,上海区域,72010889450039800000,2,46,46.0,0.028256,1.0


In [0]:
df_station = df5
for i in tqdm(df_station.index):
    if df_station.at[i,'累计销量占比'] < 0.4:
        df_station.at[i,'区域畅销款'] = 'S'
    elif df_station.at[i,'累计销量占比']<0.7:
	    df_station.at[i,'区域畅销款'] = 'A'
    else:
        df_station.at[i,'区域畅销款'] = 'B'
	

100%|██████████| 53863/53863 [00:01<00:00, 46128.83it/s]


In [0]:
df_station.head()

,区域,id,品类,销量,销量(区域),累计销量占比,区域排名,区域畅销款
0,郑州区域,75063654370028000000,2,122,122.0,0.075123,1.0,S
1,成都区域,5302155930035000000,2,10,10.0,0.007812,1.0,S
2,北京区域,5143656410003980000,9,20,20.0,0.049261,1.0,S
3,南京区域,75404815510042000000,9,6,6.0,0.010582,1.0,S
4,南京区域,75442708780039500000,2,44,44.0,0.020706,2.0,S


In [0]:
df10 = df2.sort_values(by='销量',ascending=False)
df10.drop_duplicates(subset='id',keep='first',inplace=True)

In [0]:
store = list(set(df2.店铺))
for i in tqdm(store):
    df11 = df10[df10['店铺']==i]
    n = 1
    for m in df11.index:
        df10.at[m,'店铺排名'] = n
        n += 1

100%|██████████| 203/203 [00:02<00:00, 100.43it/s]


In [0]:
df10.head()

,区域,店铺id,店铺,品类,id,销量,销量(sum),库存,全国畅销款,区域畅销款,店铺排名,区域排名,全国排名
548745,贵阳区域,XF15,安顺店,15,000000M0160000300010,3160,4614,3719208,NaN,NaN,1.0,NaN,NaN
472006,成都区域,CQ33,成都万象城店,14,78833086760116000000,1120,2170,30,NaN,NaN,1.0,NaN,NaN
305047,青岛区域,YT01,烟台大悦城店,2,75302189450045000000,906,1184,2360,NaN,NaN,1.0,NaN,NaN
517148,贵阳区域,XF21,贵阳TPMALL店,2,75404889180048000000,746,1116,89394,NaN,NaN,1.0,NaN,NaN
157832,成都区域,CQ33,成都万象城店,8,75086981750059000000,552,2088,2700,NaN,NaN,2.0,NaN,NaN


In [0]:
for i in tqdm(df.index):
    df.at[i,'全国畅销款'] = df_type[df_type['id']==df.at[i,'id']]['全国畅销款'].values[0]
    df.at[i,'全国排名'] = df_type[df_type['id']==df.at[i,'id']]['全国排名'].values[0]
    df.at[i,'区域畅销款'] = df_station[df_station['id']==df.at[i,'id']]['区域畅销款'].values[0]
    df.at[i,'区域排名'] = df_station[df_station['id']==df.at[i,'id']]['区域排名'].values[0]
    df.at[i,'店铺排名'] = df10[df10['id']==df.at[i,'id']]['店铺排名'].values[0]
df.to_excel('/home/wcjb/下载/最终版.xlsx')

In [0]:
df.to_excel('/home/wcjb/下载/最终版.xlsx')

In [0]:
df

,区域,店铺id,店铺,品类,id,销量,销量(sum),库存,全国畅销款,区域畅销款,店铺排名,区域排名,全国排名
0,重庆区域,CQ12,重庆西城天街店,8,3012860810002980000,2,24,2,B,S,209.0,1623.0,12470.0
1,南京区域,NJ04,合肥包河万达店,2,5168188480000420000,2,12,2,B,A,339.0,1974.0,19565.0
2,青岛区域,WF01,潍坊泰华城店,2,5143709110003280000,2,192,1,B,S,30.0,133.0,1836.0
3,大连区域,DL03,大连亿合城店,2,75442786380035000000,2,22,0,B,S,264.0,652.0,13158.0
4,北京区域,BJ04,北京崇文门新世界店,1,5560407590000180000,2,4,2,B,B,311.0,2672.0,33783.0
5,福州区域,QZ01,泉州星光耀万达广场店,2,75327737180039000000,2,1460,14,S,S,3.0,14.0,125.0
6,北京区域,BJ07,北京密云万象汇店,1,75342916470030000000,2,66,1,B,S,2.0,292.0,5159.0
7,重庆区域,CQ18,重庆东原店,1,5202031590001380000,2,96,2,A,S,61.0,281.0,3670.0
8,上海区域,SH09,上海凯德晶萃广场店,6,75117317840039000000,4,398,10,S,S,5.0,70.0,841.0
9,郑州区域,BT01,包头苏宁广场店,2,75063943140038000000,2,976,17,A,S,126.0,16.0,258.0
